In [44]:
import numpy as np
import matplotlib.pyplot as plt
import random

# defining simple lattice functions to place n particles in a box of length L 

def simple_lattice_2D(L,n):
     positions = []
     N = int(np.sqrt(n))
     separation = L / N
     for i in range(0,N):
            for j in range(0,N):
                x = (i + 0.5) * separation
                y = (j + 0.5) * separation
                positions.append(list([x,y]))
     return positions


def simple_lattice_3D(L,n):
    positions = []
    N = int(np.cbrt(n))
    separation = L / N
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                x = (i + 0.5) * separation
                y = (j + 0.5) * separation
                z = (k + 0.5) * separation
                positions.append((x,y,z))
    return positions

def particle_diameter_3D(L, n, vol_fraction):
    volume_per_particle = (L**3 * vol_fraction) / n
    diameter = 2*(3 * volume_per_particle / (4 * np.pi))**(1/3)
    return diameter
def particle_diameter_2D(L,n, area_fraction):
    area_per_particle = (L**2 * area_fraction) / n
    diameter = 2*(area_per_particle / np.pi)**(1/2)
    return diameter

x_coords, y_coords = zip(*simple_lattice_2D(1,100))
plt.figure()
plt.plot(x_coords, y_coords, 'o')
plt.savefig('simple_lattice_2D')

def monte_carlo_2D_np(positions, displacement, L, diameter):
    positions = np.array(positions, dtype=float)  # ensure numeric array
    N = len(positions)
    
    # Pick a random particle
    particle = random.randint(0, N - 1)
    
    # Copy positions for trial move
    new_positions = positions.copy()
    
    # Random displacement along x or y
    axis = random.randint(0, 1)       # 0=x, 1=y
    direction = 1 if random.randint(0, 1) == 0 else -1
    new_positions[particle, axis] += direction * displacement
    
    # Apply periodic boundary conditions
    new_positions[particle] %= L
    
    # Compute distances from the moved particle to all others
    diffs = new_positions - new_positions[particle]         # shape (N, 2)
    
    # Minimum image convention for periodic boundaries
    diffs = diffs - L * np.round(diffs / L)
    
    distances = np.linalg.norm(diffs, axis=1)
    distances[particle] = np.inf  # ignore self-distance
    
    # Check overlap
    if np.any(distances < diameter):
        return positions  # reject move
    else:
        return new_positions  # accept move
    
L = 4
n = 4 
vol_fraction = 0.1
no_density = n / (L**2)
diameter = particle_diameter_2D(L, n, vol_fraction)



In [35]:
# create stepwise function to move random particle in random direction by displacement amount specified
#also account for the periodic boundary conditions inside the movement 
import random

def monte_carlo_2D(positions, displacement, L, diameter):
    positions = np.array(positions, dtype=float)  # ensure numeric array
    N = len(positions)
    
    # Pick a random particle
    particle = random.randint(0, N - 1)
    
    # Copy positions for trial move
    new_positions = positions.copy()
    
    #random angle for movement and then calculate x and y changes
    angle = random.uniform(0, 2 * np.pi)
    x_change = displacement * np.cos(angle)
    y_change = displacement * np.sin(angle)
    new_positions[particle, 0] += x_change
    new_positions[particle, 1] += y_change
    # Apply periodic boundary conditions
    new_positions[particle] %= L
    
    # Compute distances from the moved particle to all others
    diffs = new_positions - new_positions[particle]         # shape (N, 2)
    
    # Minimum image convention for periodic boundaries
    diffs = diffs - L * np.round(diffs / L)
    
    distances = np.linalg.norm(diffs, axis=1)
    distances[particle] = np.inf  # ignore self-distance
    
    # Check overlap
    if np.any(distances < diameter):
        return positions  # reject move
    else:
        return new_positions  # accept move



In [36]:
#list of provided parameters - good starting point


L = 1
n = 100
vol_fraction = 0.72
no_density = n / (L**2)
diameter = particle_diameter_2D(L, n, vol_fraction)
pos_0 = simple_lattice_2D(L,n)

pos_new = monte_carlo_2D(pos_0, 0.0001, L, diameter)

if np.array_equal(pos_new, pos_0):
    print("Move rejected")
else:
    print("Move accepted")
  
#particles free to move with monte carlo function for 2D
#code to find the optimum displacement value to achieve 25% - 50% acceptance rate
acceptance_count = 0
pos_ref = pos_0
for i in range(0,1000):
    pos_new = monte_carlo_2D(pos_ref, 0.0043,L, diameter)
    if np.array_equal(pos_new, pos_ref):
        pass #move rejected
    else:
        acceptance_count += 1 #move accepted
        pos_ref = pos_new
print(acceptance_count)

#gives rougly 39% acceptance rate for the suggested parameters

Move accepted
379


In [37]:
#define total energy calculation function

def total_energy(positions, L, diameter):
    positions = np.array(positions, dtype=float)
    N = len(positions)
    total_energy = 0.0
    
    for i in range(N):
        for j in range(i + 1, N):
            # Compute distance with minimum image convention
            diff = positions[i] - positions[j]
            diff = diff - L * np.round(diff / L)
            distance = np.linalg.norm(diff)
            
            # Simple hard-sphere potential
            if distance < diameter:
                total_energy += np.inf  # Infinite energy for overlap
            else:
                total_energy += 0.0  # No interaction otherwise
                
    return np.array(total_energy)

# define function to calculate the change in energy after a single move
def energy_change(positions, initial_energy, new_positions):
    new_coords = np.any(new_positions != positions, axis=1)
    moved_particle = np.where(new_coords)[0][0]
    new_energy = initial_energy
    for i in range(0,len(positions)):
        if i == moved_particle:
            continue  # Skip self-interaction
        # Compute distance with minimum image convention
        diff_old = positions[moved_particle] - positions[i]
        diff_old = diff_old - L * np.round(diff_old / L)
        distance_old = np.linalg.norm(diff_old)
        
        diff_new = new_positions[moved_particle] - new_positions[i]
        diff_new = diff_new - L * np.round(diff_new / L)
        distance_new = np.linalg.norm(diff_new)
        
        # Simple hard-sphere potential
            
        if distance_new < diameter and distance_old >= diameter:
            new_energy = np.inf + initial_energy # add infinite energy if moving into overlap
        if distance_new >= diameter and distance_old < diameter:
            new_energy = -np.inf + initial_energy # Remove infinite energy if moving out of overlap
        else:
            new_energy = 0.0 + initial_energy # no overlap either before or after
    return new_energy

In [43]:
#test initial energy and energy change functions
initial_energy = total_energy(pos_0, L, diameter)
pos_new = monte_carlo_2D(pos_0, 0.0043,L,diameter)
delta_E = energy_change(np.array(pos_0), initial_energy, pos_new)
print(initial_energy)
print(delta_E)
#i think this works, as E should always be 

0.0
0.0


In [39]:
#defining the metropolis algorithm - will have to change the monte carlo to accept even if center of spheres are within a radius
def metropolis(positions,initial_energy, delta_E_func, stepwise_function, displacement, L, diameter, k_B, T):
    positions = np.array(positions)
    new_positions = stepwise_function(positions, displacement, L , diameter)
    delta_E = delta_E_func(positions, initial_energy, new_positions)
    if delta_E > 0:
        n == random.rand(0,1)
        if n < np.exp(-delta_E / (k_B * T)):#how do we find T
            return  new_positions
        else:
            return positions
    else:
        return positions
# this should allow you to input the initial energy, positions, displacement, L Boltzman constant and temperature 
#then will perform a montecarlo step to the system to move particle
#calculate the energy after the move and accept if energy is lowered
#if energy is the same or more then it is accepted if the metropolis function is less than the random number generated


In [65]:
#define the pair correlation function
#try delta_r as approx 0.1 times the diameter as initial trial
def pair_correlation_2D(positions, diameter, L, bins=100):
    positions = np.array(positions)
    N = len(positions) # no.of particles
    hist = np.zeros(bins, dtype=float) # create histogram array for each bin
    delta_r = (L / 2) / bins  # bin width
    
    for i in range(N):
        for j in range(i + 1, N):
            # Compute distance with minimum image convention
            diff = positions[i] - positions[j]
            diff = diff - L * np.round(diff / L)
            distance = np.linalg.norm(diff)
            
            # Determine which bin this distance falls into
            bin_index = int(distance / delta_r)
            if bin_index < bins:
                hist[bin_index] += 1  # count each pair once
            
    r = ((np.arange(bins) + 0.5) * delta_r) / diameter # midpoint of each bin for average value of r 
    area_density = N / (L**2)
    n_ideal = 2 * np.pi * r * delta_r * area_density * N # ideal gas distribution

    g_r = hist / n_ideal
    return r, g_r




In [79]:
#trial of pair correlation function
N = 100
L = 1
area_fraction = 0.68
#equilibration run 
diameter = particle_diameter_2D(L, N, area_fraction)
pos_0 = simple_lattice_2D(L,N)
pos = pos_0
r, g_r = pair_correlation_2D(pos,diameter, L)
r_initial, g_r_initial = pair_correlation_2D(pos_0, diameter, L) # initial position before equilibrating
#run monte carlo 10**4 times
g_r_total_eq = []

for i in range(0, 4*10 **4 + 1):
    pos = monte_carlo_2D(pos, 0.0043, L, diameter)
    if i % 10000 == 0:
        r, g_r = pair_correlation_2D(pos,diameter, L, bins = 100)
        g_r_total_eq.append(g_r)

x2_coords = r
y1_coords = g_r_total[0]
y2_coords = g_r_total[1]
y3_coords = g_r_total[2]
y4_coords = g_r_total[3]
y5_coords = g_r_total[4]
plt.figure()
plt.plot(x2_coords, y1_coords, label='After 0 moves')
plt.plot(x2_coords, y2_coords, label='After 10000 moves')
plt.plot(x2_coords, y3_coords, label='After 20000 moves')
plt.plot(x2_coords, y4_coords, label='After 30000 moves')
plt.plot(x2_coords, y5_coords, label='After 40000 moves')
plt.savefig('equillibration_2D.png')

plt.figure()
x_coords, y_coords = zip(*pos)
plt.figure()
plt.plot(x_coords, y_coords, 'o')
plt.savefig('simple_lattice_2D_equillibrates.png')

In [80]:
#averagine the g_r values after equillibration
#use position from previous code block
g_r_total = []
#run through the monte carlo steps again    
for i in range(0, 5*10 **5 + 1):
    pos = monte_carlo_2D(pos, 0.0043, L, diameter)
    if i % 1000 == 0:
        r, g_r = pair_correlation_2D(pos,diameter, L, bins = 100)
        g_r_total.append(g_r)

x2_coords_new = r
y_coords_new = np.mean(g_r_total, axis=0)

#don't run any more code in this box as takes approx 20 seconds to run


In [ ]:
#make a pretty graph for 2D average g(r)
#calculate some errors too

g_r_std_err = np.std(g_r_total, axis = 0) / np.sqrt(len(g_r_total))
x_err  = (L/2) / 200 # bin width / 2
#could try to find a model to fit - might be a decaying exponential? / don't know how to do this though

plt.figure()
plt.plot(x2_coords_new, y_coords_new, label = 'average g(r)', color = 'black', linewidth = 0.5)
plt.errorbar(x2_coords_new, y_coords_new, yerr = g_r_std_err, xerr = x_err, fmt = 'o',color = 'blue', ecolor = 'navy', capsize = 2, markersize = 1, label = 'average g(r)')
plt.xlabel('r / $\sigma$')
plt.ylabel('g(r)')
plt.legend()
plt.savefig('average_g(r).png')

<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_31586/1375519868.py:10: SyntaxWarning: invalid escape sequence '\s'
  plt.xlabel('r / $\sigma$')


In [100]:
#extend the functions to 3D
#define the monte carlo function for stepwise in 3D - still using the hard sphere model so no overlaps

def monte_carlo_3D(positions, displacement, L, diameter):
    positions = np.array(positions, dtype=float)  # ensure numeric array
    N = len(positions)
    
    # Pick a random particle
    particle = random.randint(0, N - 1)
    
    # Copy positions for trial move
    new_positions = positions.copy()
    
    # find 2 random angles for movement in 3D
    cos_theta = np.random.uniform(-1, 1)
    theta = np.arccos(cos_theta)
    phi = random.uniform(0, 2 * np.pi)
    x_change = displacement * np.sin(theta) * np.cos(phi)
    y_change = displacement * np.sin(theta) * np.sin(phi)
    z_change = displacement * cos_theta
    new_positions[particle, 0] += x_change
    new_positions[particle, 1] += y_change
    new_positions[particle, 2] += z_change
    # Apply periodic boundary conditions
    new_positions[particle] %= L
    # Compute distances from the moved particle to all others
    diffs = new_positions - new_positions[particle]         # shape (N, 3)
    # Minimum image convention for periodic boundaries
    diffs = diffs - L * np.round(diffs / L)
    distances = np.linalg.norm(diffs, axis=1)
    distances[particle] = np.inf  # ignore self-distance    
    # Check overlap
    if np.any(distances < diameter):
        return positions  # reject move
    else:
        return new_positions  # accept move
    
#define the pair correlation function for 3D
def pair_correlation_3D(positions, diameter, L, bins=100):
    positions = np.array(positions)
    N = len(positions) # no.of particles
    hist = np.zeros(bins, dtype=float) # create histogram array for each bin
    delta_r = (L / 2) / bins  # bin width
    
    for i in range(N):
        for j in range(i + 1, N):
            # Compute distance with minimum image convention
            diff = positions[i] - positions[j]
            diff = diff - L * np.round(diff / L)
            distance = np.linalg.norm(diff)
            
            # Determine which bin this distance falls into
            bin_index = int(distance / delta_r)
            if bin_index < bins:
                hist[bin_index] += 1  # count each pair once
            
    r = ((np.arange(bins) + 0.5) * delta_r) / diameter # midpoint of each bin for average value of r 
    volume_density = N / (L**3)
    n_ideal = 4 * np.pi * r**2 * delta_r * volume_density * N # ideal gas distribution

    g_r = hist / n_ideal
    return r, g_r


In [ ]:
#testing out the 3D pair correlation function
L = 1
N = 1000
positions = simple_lattice_3D(L,N)
vol_fraction = 0.68
diameter = particle_diameter_3D(L, N, vol_fraction)
pos_3D = positions.copy()
r_3D, g_r_3D = pair_correlation_3D(positions, diameter, L, bins=100)
g_r_total_3D_eq = []
#equillibration run
for i in range(0, 4*10 **4 + 1):
    pos_3D = monte_carlo_3D(pos_3D, 0.0043, L, diameter)
    if i % 10000 == 0:
        r_3D_new, g_r_new_3D = pair_correlation_3D(pos_3D,diameter, L, bins = 100)
        g_r_total_3D_eq.append(g_r_new_3D)
x_coords_3D = r_3D
y_coords_3D_eq = np.mean(g_r_total_3D_eq, axis=0)
plt.figure()
plt.plot(x_coords_3D, y_coords_3D_eq)
plt.savefig('equillibration_3D.png')

#continue from equillibration to get average g(r)
g_r_total_3D = []
#run through the monte carlo steps again
for i in range(0, 5*10 **5 + 1):
    pos_3D = monte_carlo_3D(pos_3D, 0.0043, L, diameter)
    if i % 1000 == 0:
        r_3D_new, g_r_new_3D = pair_correlation_3D(pos_3D,diameter, L, bins = 100)
        g_r_total_3D.append(g_r_new_3D)


In [ ]:
y_coords_3D = np.mean(g_r_total_3D, axis=0)
plt.figure()
plt.plot(x_coords_3D, y_coords_3D)
plt.savefig('average_g(r)_3D.png')